# DDPM

## Set global variable

In [6]:
# Path to dataset
DATASET = "dataset/cats/"
# DATASET = "dataset/lfw/"
DATASET_NAME = DATASET.rstrip('/').split('/')[-1]

# Hyperparameter for training
IMG_SIZE = 64
CHANNELS_IMG = 3
BATCH_SIZE = 16

LEARNING_RATE = 1e-4  #0.0001
# LEARNING_RATE = 1e-5  #0.00001
NUM_EPOCHS = 100

print(DATASET_NAME)

cats


## Count and set number of image variable

In [7]:
import os
import glob

def count_image_files(folder_path, extensions=['.jpg', '.jpeg', '.png', '.gif']):
    count = 0
    for extension in extensions:
        search_pattern = os.path.join(folder_path, '*' + extension)
        count += len(glob.glob(search_pattern))
    return count


In [8]:
NUM_IMAGES = count_image_files(DATASET)

print(f"Number of image files in {DATASET}: {NUM_IMAGES}")

Number of image files in dataset/cats/: 15747


## Set agnostic code

In [4]:
import torch

# agnostic code to detect the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


## Load images to tensor and normalize images

In [5]:
import torchvision.transforms as transforms
from PIL import Image

# Get the list of image filenames
image_filenames = os.listdir(DATASET)

# Define the image transformations
transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    # transforms.RandomHorizontalFlip(),
    transforms.ToTensor(), # convert PIL image to tensor and scales data into [0,1] 
    # transforms.Lambda(lambda t: (t * 2) - 1) # Scale between [-1, 1] 
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]), # Scale between [-1, 1] by (input[channel] - mean[channel]) / std[channel]
])

# Create a list to store the transformed images
transformed_images = []

# Iterate over the first num_images filenames and transform the corresponding images
for i, filename in enumerate(image_filenames[:NUM_IMAGES]):
    # Load the image
    img_path = os.path.join(DATASET, filename)
    image = Image.open(img_path)

    # Apply the transformations
    transformed_image = transform(image)

    # Append the transformed image to the list
    transformed_images.append(transformed_image)

# Convert the list of transformed images to a PyTorch tensor
transformed_images = torch.stack(transformed_images)

print(f'Loaded data: {transformed_images.shape}')

Loaded data: torch.Size([12876, 3, 64, 64])


## Separate batch size 

In [6]:
from torch.utils.data import DataLoader

data_loader = DataLoader(transformed_images, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)

# print first batch.shape
data_iter = iter(data_loader)
print(next(data_iter).shape)

torch.Size([16, 3, 64, 64])


## Train model and save weight and log 

## training process
parameter base on DDPM and unet papers https://arxiv.org/pdf/1505.04597v1.pdf

In [6]:
import time
from modules.ddpm import Diffusion, UNet

import torch.optim as optim
import torch.nn as nn
import torchvision

from torch.utils.tensorboard import SummaryWriter

model = UNet().to(device)
optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE)
mse = nn.MSELoss()
diffusion = Diffusion(img_size=IMG_SIZE, device=device)
writer = SummaryWriter(os.path.join(f"logs/{DATASET_NAME}","DDPM"))

l = len(data_loader)

time_use = 0

for epoch in range(NUM_EPOCHS):

    lossMean = 0

    # use time for time measurement
    start_time = time.time()

    model.train()
    
    for batch_idx, images in enumerate(data_loader):
        images = images.to(device)
        t = diffusion.sample_timesteps(images.shape[0]).to(device)
        x_t, noise = diffusion.noise_images(images, t)
        predicted_noise = model(x_t, t)
        loss = mse(noise, predicted_noise)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # sum lose
        lossMean += loss

        # add log at first 
        if epoch == 0 and batch_idx == 0:
            writer.add_scalar("Loss/MSE", lossMean, global_step=epoch)
            writer.add_scalar("traing time", time_use, epoch)

    end_time = time.time()
    epoch_time = end_time - start_time

    time_use += epoch_time 

    epoch_index = epoch+1

    # calculate mean value
    lossMean = lossMean / len(data_loader.dataset)

    print(
        f"Epoch [{epoch_index}/{NUM_EPOCHS}] Batch {batch_idx+1}/{len(data_loader)} Using Time: {epoch_time:.4f}\
            Loss: {lossMean:.4f}"
    )

    # tensorboard
    writer.add_scalar("Loss/MSE", lossMean, global_step=epoch+1)
    writer.add_scalar("traing time", time_use, epoch_index)

    weight_path = os.path.join("weights", f'{DATASET_NAME}', "DDPM")
    os.makedirs(weight_path, exist_ok=True)
    torch.save(model.state_dict(), os.path.join("weights", f'{DATASET_NAME}', "DDPM", f"{epoch_index}.pt"))

    model.eval()
    with torch.no_grad():
        x = diffusion.sample(model, n=32).type(dtype=torch.float32)
        img_grid = torchvision.utils.make_grid(x[:32], normalize=True)
        writer.add_image("All/Gen", img_grid, global_step=epoch_index)


/home/nice/mambaforge/lib/python3.10/site-packages/torch/nn/modules/conv.py:459: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at /opt/conda/conda-bld/pytorch_1682343995026/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv2d(input, weight, bias, self.stride,


Epoch [1/100] Batch 804/804 Using Time: 206.7065            Loss: 0.0072


999it [02:43,  6.10it/s]


Epoch [2/100] Batch 804/804 Using Time: 212.6587            Loss: 0.0025


999it [02:44,  6.07it/s]


...

999it [02:45,  6.03it/s]


Epoch [99/100] Batch 804/804 Using Time: 213.2356            Loss: 0.0009


999it [02:44,  6.07it/s]


Epoch [100/100] Batch 804/804 Using Time: 211.6610            Loss: 0.0009


999it [02:44,  6.08it/s]


# Resume Training

In [7]:
def load_weight( model, weight_path="", index=int):
    # Load the saved weights
    model.load_state_dict(torch.load(f'{weight_path}/{index}.pt'))

## get logs latest training time to count on this number

In [8]:
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
import matplotlib.pyplot as plt

# Function to get data from event accumulator
def get_data(log_dir):
    event_acc = EventAccumulator(log_dir)
    event_acc.Reload()

    tags = event_acc.Tags()['scalars']
    selected_tag = 'traing time'
    tag_values = event_acc.Scalars(selected_tag)
    
    steps = [x.step for x in tag_values]
    values = [x.value for x in tag_values]
    return steps, values

steps, values = get_data(f'logs/{DATASET_NAME}/DDPM/')

print(len(steps))
print(len(values))

print(steps.index(steps[-1]), "value:", steps[-1] )
print(values.index(values[-1]), "value:", values[-1] )
# print(values.index(values[451]), "value:", values[451] )

144
144
143 value: 143
143 value: 30045.22265625


## resume

In [9]:
import time
from modules.ddpm import Diffusion, UNet

import torch.optim as optim
import torch.nn as nn
import torchvision

from torch.utils.tensorboard import SummaryWriter


# resume number
start = steps[-1]
stop = 500 

model = UNet().to(device)

# load weight
load_weight(model=model, weight_path=f'weights/{DATASET_NAME}/DDPM/', index=start)

optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE)
mse = nn.MSELoss()
diffusion = Diffusion(img_size=IMG_SIZE, device=device)
writer = SummaryWriter(os.path.join(f"logs/{DATASET_NAME}","DDPM"))

# l = len(data_loader)

# get trianing time to resume counting number
time_use = values[-1]

for epoch in range(start,stop):

    lossMean = 0

    # use time for time measurement
    start_time = time.time()

    model.train()
    
    for batch_idx, images in enumerate(data_loader):
        images = images.to(device)
        t = diffusion.sample_timesteps(images.shape[0]).to(device)
        x_t, noise = diffusion.noise_images(images, t)
        predicted_noise = model(x_t, t)
        loss = mse(noise, predicted_noise)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # sum lose
        lossMean += loss

    end_time = time.time()
    epoch_time = end_time - start_time

    time_use += epoch_time 

    epoch_index = epoch+1

    # calculate mean value
    lossMean = lossMean / len(data_loader.dataset)

    print(
        f"Epoch [{epoch_index}/{stop}] Batch {batch_idx+1}/{len(data_loader)} Using Time: {epoch_time:.4f}\
            Loss: {lossMean:.4f}"
    )

    # tensorboard
    writer.add_scalar("Loss/MSE", lossMean, global_step=epoch+1)
    writer.add_scalar("traing time", time_use, epoch_index)

    weight_path = os.path.join("weights", f'{DATASET_NAME}', "DDPM")
    os.makedirs(weight_path, exist_ok=True)
    torch.save(model.state_dict(), os.path.join("weights", f'{DATASET_NAME}', "DDPM", f"{epoch_index}.pt"))

    model.eval()
    with torch.no_grad():
        x = diffusion.sample(model, n=32).type(dtype=torch.float32)
        img_grid = torchvision.utils.make_grid(x[:32], normalize=True)
        writer.add_image("All/Gen", img_grid, global_step=epoch_index)


/home/nice/mambaforge/lib/python3.10/site-packages/torch/nn/modules/conv.py:459: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at /opt/conda/conda-bld/pytorch_1682343995026/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv2d(input, weight, bias, self.stride,


Epoch [144/500] Batch 804/804 Using Time: 212.2353            Loss: 0.0009


999it [02:37,  6.35it/s]


Epoch [145/500] Batch 804/804 Using Time: 203.1017            Loss: 0.0009


999it [02:37,  6.33it/s]


...

999it [02:39,  6.25it/s]


Epoch [499/500] Batch 804/804 Using Time: 206.9759            Loss: 0.0008


999it [02:39,  6.25it/s]


Epoch [500/500] Batch 804/804 Using Time: 206.9300            Loss: 0.0008


999it [02:40,  6.23it/s]


# Measure FID

use this implementation: https://github.com/mseitzer/pytorch-fid/tree/master

## define FID measurement function 

In [5]:
import subprocess
import torch
from torchvision.utils import save_image
import re
import os
import gc
from modules.ddpm import  Diffusion


# Create a function to run the FID script
def run_fid(real_path, gen_path):
    command = ["python", "-m", "pytorch_fid", real_path, gen_path]

    output = subprocess.run(command, capture_output=True, text=True)

    # Extract the FID score using regular expressions
    output = output.stdout

    fid_score_match = re.search(r"FID:\s+(-?\d+\.\d+)", output)
    
    if fid_score_match:
        fid_score = float(fid_score_match.group(1))
        
        return fid_score

    else:
        print("FID score not found in the output.")

def generate_images(model, sample_size=100*8, batch_size=8, device="cuda", real_path=f"dataset/{DATASET_NAME}/", gen_path=f"generated_images/{DATASET_NAME}/DCGAN"):

    diffusion = Diffusion(img_size=IMG_SIZE, device=device)
    
    # Set the model to evaluation mode
    model.eval()

    # Create a folder for generated images if it doesn't exist
    os.makedirs(gen_path, exist_ok=True)

    # Generate images
    with torch.no_grad():
        for i in range(0, sample_size, batch_size):
            
            x = diffusion.sample(model, n=batch_size).type(dtype=torch.float32)

            for j in range(batch_size):
                save_image(x[j], f"{gen_path}/{i+j}.png", normalize=True)
        
    # model = None
    # gc.collect() 

    with torch.no_grad():
        del model
        torch.cuda.empty_cache()



## Load weight of model to measure FID score

In [10]:
def load_weight( model, weight_path="", index=int):
    # Load the saved weights
    model.load_state_dict(torch.load(f'{weight_path}/{index}.pt'))
    

In [8]:
from modules.ddpm import UNet
from pathlib import Path
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter(f'logs/{DATASET_NAME}/DDPM/')

# Specify the directory path
directory_path = Path(f'weights/{DATASET_NAME}/DDPM/')

# Get the list of files ending with ".pth"
# file_list = list(directory_path.glob('*.pt'))

sample_size = 5000

first_number = 1
last_number = 500
frequency = 8  # Specify the frequency of numbers in between

numbers = [first_number] + [first_number + (last_number - first_number) // (frequency + 1) * i for i in range(1, frequency + 1)] + [last_number]

for i in numbers:

    index_weight = i

    # model = UNet().to(device)

    # load_weight(model, weight_path=directory_path, index=index_weight)

    # generate_images(model, sample_size, batch_size=64, device=f'{device}', gen_path=f'gen/{DATASET_NAME}/DDPM/{index_weight}')

    # FID measurement
    fid_score = run_fid(f"{DATASET}", f"gen/{DATASET_NAME}/DDPM/{index_weight}")
    print(f'Epoch: {index_weight} FID score: {fid_score}')
    writer.add_scalar("Metrics/FID Score", fid_score, index_weight)


Epoch: 1 FID score: 263.81401218110216
Epoch: 8 FID score: 308.22987261783453
Epoch: 15 FID score: 178.0814243691487
Epoch: 22 FID score: 292.82191242531616
Epoch: 29 FID score: 240.96353239079892
Epoch: 36 FID score: 204.16533503679705
Epoch: 43 FID score: 252.44504759055448
Epoch: 50 FID score: 219.4068309436546
Epoch: 57 FID score: 215.58892887369643
Epoch: 64 FID score: 141.97673180449073
Epoch: 71 FID score: 167.43182430718068
Epoch: 78 FID score: 232.9618261505923
Epoch: 85 FID score: 116.75266722945042
Epoch: 92 FID score: 206.89526886624148
Epoch: 99 FID score: 103.21029417948219


# subprocess

In [ ]:
import subprocess

index = 1
# Define the command to be executed
command = [
    "python",
    "modules/ddpm_sample.py",
    "--sample-size",
    "900",
    "--batch-size",
    "50",
    "--gen-path",
    f"gen/{DATASET_NAME}/DDPM/{index}",
    "--weight-path",    
    f"weights/{DATASET_NAME}/DDPM/{index}.pt"
]

# Run the command and capture the output
try:
    output = subprocess.check_output(command)
    output = output.decode("utf-8")  # Decode the byte output to string (if needed)
    print(output)
except subprocess.CalledProcessError as e:
    print("Command execution failed:", e)


In [10]:
import subprocess

first_number = 1
last_number = 500
frequency = 8  # Specify the frequency of numbers in between

numbers = [first_number] + [first_number + (last_number - first_number) // (frequency + 1) * i for i in range(1, frequency + 1)] + [last_number]

# print(len(epochs))
samples_n = 5000

for index in numbers:
    gen_n = count_image_files(f"gen/{DATASET_NAME}/DDPM/{index}")

    if gen_n < samples_n:
        remain = samples_n - gen_n
        print("Epoch:",index,"Remaining",remain,"images to generate")

        command = [
            "python",
            "modules/ddpm_sample.py",
            "--sample-size",
            f"{remain}",
            "--batch-size",
            "50",
            "--gen-path",
            f"gen/{DATASET_NAME}/DDPM/{index}",
            "--weight-path",
            f"weights/{DATASET_NAME}/DDPM/{index}.pt"
        ]

        try:
            output = subprocess.check_output(command)
            output = output.decode("utf-8")  # Decode the byte output to string (if needed)
            print(output)
        except subprocess.CalledProcessError as e:
            print("Command execution failed:", e)
        
    else:
        print("Epoch:",index,"Done")

Epoch: 1 Done
Epoch: 56 Remaining 5000 images to generate


0it [00:00, ?it/s]/home/nice/mambaforge/lib/python3.10/site-packages/torch/nn/modules/conv.py:459: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at /opt/conda/conda-bld/pytorch_1682343995026/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv2d(input, weight, bias, self.stride,
898it [04:43,  3.26it/s]

# Count learnable parameter in model

In [3]:
model = UNet().to("cuda")


pytorch_total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

In [4]:
print(pytorch_total_params)

22291587
